In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk

In [2]:
from src.conditional_ngram_models import BigramSpellCorrector, TrigramSpellCorrector
from src.ngram_models import BigramModel, START_TOKEN, END_TOKEN, UNKNOWN_TOKEN, TrigramModel, BaseNgramModel

## The Dataset

In [3]:
nltk.download('punkt')
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to /home/dimits/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /home/dimits/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [4]:
# TODO: we should probably select similar text collections
gutenberg_corpus = nltk.corpus.gutenberg.fileids()                                 #Get all the files
gutenberg_corpus

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [5]:
combined_text = ""             
for file_id in gutenberg_corpus:                                        # Combine the text from all files
    combined_text += nltk.corpus.gutenberg.raw(file_id)

print(combined_text[:500])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died t


In [6]:
combined_text = combined_text.lower()                              #Convert to lowercase

In [7]:
combined_text[:500]

"[emma by jane austen 1816]\n\nvolume i\n\nchapter i\n\n\nemma woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; and had lived nearly twenty-one years in the world\nwith very little to distress or vex her.\n\nshe was the youngest of the two daughters of a most affectionate,\nindulgent father; and had, in consequence of her sister's marriage,\nbeen mistress of his house from a very early period.  her mother\nhad died t"

In [8]:
import re

def remove_special_chars(text):
   text = text.replace('[', '')
   text = text.replace(']', '')
   text = text.replace('\n', ' ')
   text = re.sub(r'[^a-zA-z.?!\']', ' ', text)                     #Remove these characters   

   return text

In [9]:
combined_text = remove_special_chars(combined_text)
combined_text[:500]

"emma by jane austen       volume i  chapter i   emma woodhouse  handsome  clever  and rich  with a comfortable home and happy disposition  seemed to unite some of the best blessings of existence  and had lived nearly twenty one years in the world with very little to distress or vex her.  she was the youngest of the two daughters of a most affectionate  indulgent father  and had  in consequence of her sister's marriage  been mistress of his house from a very early period.  her mother had died too"

In [10]:
len(combined_text.split())                     

2119883

In [11]:
len(combined_text)                             # How many characters

11793056

In [12]:
combined_text[11792000:]

"n the sea  the universe  the stars there in the     heavens   urging slowly  surely forward  forming endless  and waiting ever more  forever more behind.       good bye my fancy!  good bye my fancy! farewell dear mate  dear love! i'm going away  i know not where  or to what fortune  or whether i may ever see you again  so good bye my fancy.  now for my last  let me look back a moment  the slower fainter ticking of the clock is in me  exit  nightfall  and soon the heart thud stopping.  long have we lived  joy'd  caress'd together  delightful!  now separation  good bye my fancy.  yet let me not be too hasty  long indeed have we lived  slept  filter'd  become really blended     into one  then if we die we die together   yes  we'll remain one   if we go anywhere we'll go together to meet what happens  may be we'll be better off and blither  and learn something  may be it is yourself now really ushering me to the true songs   who     knows?  may be it is you the mortal knob really undoing 

In [13]:
def tokenize_sentences(text):
    sentences = nltk.sent_tokenize(''.join(text))                  #Get the sentences
    return sentences     

In [14]:
sentences = tokenize_sentences(combined_text) 
print(len(sentences))    
print(sentences[2])  
print(sentences[57649])

96282
her mother had died too long ago for her to have more than an indistinct remembrance of her caresses  and her place had been supplied by an excellent woman as governess  who had fallen little short of a mother in affection.
i hardly understand you   replied the scientist  with a cold intensity of manner.


In [15]:
def tokenize_words(text):
    words = nltk.word_tokenize(text)
    return words

In [16]:
words = tokenize_words(combined_text)
print(len(words))
print(words[3])
print(words[-2])

2235498
austen
fancy


In [17]:
words_list = []                                    #list of all the words of sentences
for f in sentences:
    words_list.append(tokenize_words(f))                   

In [18]:
len(words_list)

96282

In [19]:
for word in words_list[1]:                     # all the words of the second sentence
    print(word) 

she
was
the
youngest
of
the
two
daughters
of
a
most
affectionate
indulgent
father
and
had
in
consequence
of
her
sister
's
marriage
been
mistress
of
his
house
from
a
very
early
period
.


In [20]:
import random
import math


random.Random(0).shuffle(words_list)
train_len = math.floor(0.6 * len(words_list))                      #Training set length(60%)
dev_len = math.floor(0.2 * len(words_list))                        #Development set length (20%)
test_len = math.floor(0.2 * len(words_list))                       #Test set length (20%)

training_set = []
development_set = []
test_set = []

for content in words_list[0:train_len]:
    training_set.append(content)
    
for content in words_list[train_len: train_len + dev_len]:
    development_set.append(content)

for content in words_list[train_len + dev_len:]:
    test_set.append(content)

In [21]:
import itertools
from collections import Counter

from nltk.util import ngrams


def _calc_ngrams(all_corpus: list[str], ngram: int) -> Counter:
    """
    Process a tokenized sentence into a list of ngrams.
    :param all_corpus: a list of all the corpus words
    :param ngram: whether the ngrams will be unigrams, bigrams etc
    :return: the counter of either unigram, bigram or trigram
    """
    unigram_counter = Counter()
    bigram_counter = Counter()
    trigram_counter = Counter()
     
    

    if ngram == 1 :
        for sentence in all_corpus:
             grams = [gram for gram in ngrams(sentence, ngram, pad_left=True, pad_right=True,
                                    left_pad_symbol=START_TOKEN, right_pad_symbol=END_TOKEN)]
             unigram_counter.update(grams)
        return unigram_counter
        
    elif ngram == 2:
        for sentence in all_corpus:
             grams = [gram for gram in ngrams(sentence, ngram, pad_left=True, pad_right=True,
                                    left_pad_symbol=START_TOKEN, right_pad_symbol=END_TOKEN)]
             bigram_counter.update(grams)
        return bigram_counter
        
    elif ngram == 3:
        for sentence in all_corpus:
             grams = [gram for gram in ngrams(sentence, ngram, pad_left=True, pad_right=True,
                                    left_pad_symbol=START_TOKEN, right_pad_symbol=END_TOKEN)]
             trigram_counter.update(grams)
        return trigram_counter
        
    return 0

In [22]:
def replace_OOV_words_train(all_corpus):
    unigram_counter = _calc_ngrams(all_corpus,1)
    OOV_words = {}

    for k, v in unigram_counter.items():
        if v < 10:
            key = k[0]
            # README: Use the symbol UNKOWN_TOKEN else the model will think it's a word
            OOV_words[key] = UNKNOWN_TOKEN                 #set the word to "UNK"

    replaced_corpus = []                          #the original corpus having the OOV words replaced by 'UNK'
    for sentence in all_corpus:
        clean_sentence = []
    
        for word in sentence:
            clean_sentence.append(OOV_words.get(word, word))
    
        replaced_corpus.append(clean_sentence)


    vocabulary = []

    for key in unigram_counter.keys():        #Iterate the unigram counter
        word = key[0]                         #get the word
        if word not in OOV_words:
            vocabulary.append(word)

    vocabulary = set(vocabulary)              #Keep unique words
    return vocabulary, replaced_corpus, OOV_words

In [23]:
vocabulary, train_corpus, OOV_words = replace_OOV_words_train(training_set)

In [24]:
def replace_OOV_words_test(all_corpus, vocabulary, oov_words):
    
    replaced_corpus = []
    for sentence in all_corpus:
        updated_sent = []

        for word in sentence:
            if (word not in vocabulary) or (word in oov_words):
                updated_sent.append(UNKNOWN_TOKEN)
            else:
                updated_sent.append(word)
                
    replaced_corpus.append(updated_sent)
    return replaced_corpus   

In [25]:
development_set = replace_OOV_words_test(development_set, vocabulary, OOV_words)
test_set = replace_OOV_words_test(test_set, vocabulary, OOV_words)

In [26]:
vocab_len = len(vocabulary)
print ("Vocabulary length: ", vocab_len)

print("Unigram's 20 most common words:")
unigram_top_20 = _calc_ngrams(train_corpus, 1).most_common(20)
for gram in unigram_top_20:
    print(gram)

print("\n")
print("Bigram's 20 most common words:")
bigram_top_20 = _calc_ngrams(train_corpus,2).most_common(20)
for gram in bigram_top_20:
    print(gram)


print("\n")
print("Trigram's 20 most common words:")
trigram_top_20 = _calc_ngrams(train_corpus,3).most_common(20)
for gram in trigram_top_20:
    print(gram)

Vocabulary length:  7371
Unigram's 20 most common words:
(('the',), 79845)
(('<UNK>',), 68339)
(('and',), 57152)
(('.',), 46364)
(('of',), 42451)
(('to',), 28992)
(('a',), 20369)
(('in',), 20160)
(('i',), 18319)
(('that',), 17192)
(('he',), 15721)
(('it',), 13330)
(('his',), 13104)
(('for',), 11883)
(('was',), 11344)
(('not',), 10796)
(('with',), 10671)
(('is',), 9864)
(('you',), 9847)
(('be',), 9719)


Bigram's 20 most common words:
(('.', '<end>'), 46118)
(('of', 'the'), 11331)
(('the', '<UNK>'), 8168)
(('<start>', 'and'), 8061)
(('<UNK>', 'and'), 6231)
(('?', '<end>'), 6100)
(('in', 'the'), 6087)
(('and', 'the'), 5242)
(('<UNK>', '<UNK>'), 5198)
(('<UNK>', '.'), 4966)
(('!', '<end>'), 4960)
(('and', '<UNK>'), 4694)
(('the', 'lord'), 4173)
(('<UNK>', 'of'), 4048)
(('<start>', 'i'), 3478)
(('of', '<UNK>'), 3370)
(('<UNK>', 'the'), 3252)
(('to', 'the'), 3233)
(('<start>', 'the'), 3103)
(('a', '<UNK>'), 2607)


Trigram's 20 most common words:
(('.', '<end>', '<end>'), 46118)
(('<start>'

In [27]:
from nltk.tokenize import sent_tokenize, TweetTokenizer


UNK_sentences = [sent_tokenize(' '.join(sentence)) for sentence in train_corpus]       #get the sentences that include UNK values
tweet_wt = TweetTokenizer()
tokenized = [tweet_wt.tokenize(' '.join(sentence)) for sentence in UNK_sentences[:50]]  # Get the first 50 sentences

## Auto-completing Language Models

##  Calculate bi-gram probability

### $ P(w_2|w_1) = \frac{C(w_1,w_2) + \alpha}{C(w_1) + \alpha \cdot|V|} $

* $ C(w_1,w_2) $ : bigram count
* $ C(w_1) $ : unigram count
* $ 0 \leq\alpha \leq1 $ :  smoothing hyper-parameter
* |V|: vocabulary size

## Bi-gram LM Cross entropy & perplexity

* $ CrossEntropy = -\frac{1}{N}\sum^{bigrams}{log_2(P(w_2|w_1))} $
 * N: Number of bigrams
* $ Perplexity = 2^{H(p)} $

## Tri-gram LM Cross entropy & perplexity

### $ P(w_3|w_1,w_2) = \frac{C(w_1,w_2,w_3) + \alpha}{C(w_1,w_2) + \alpha \cdot |V|} $

* $ C(w_1,w_2,w_3) $ : trigram count
* $ C(w_1,w_2) $ : bigram count
* $ 0 \leq\alpha \leq1 $ :  smoothing hyper-parameter
* |V|: vocabulary size

In [28]:
def cross_entropy(model: BaseNgramModel, dataset: list[list[str]]) -> float:
    """
    Calculate the cross-entropy of a language model on a given dataset.
    
    Cross-entropy measures how well the language model predicts the given dataset.
    Lower cross-entropy indicates better model performance.
    
    :param model: The n-gram language model for which cross-entropy is calculated.
    :param dataset: The dataset as a list of tokenized sentences, where each sentence is a list of strings.
    :return: The cross-entropy score as a float.
             Lower values indicate better performance in predicting the dataset.
    """
    # since la place smoothing is only involved during inference
    # we don't need to refit the model
    sum_prob = 0
    word_count = 0
    
    for sentence in dataset:       
        # since this is a full sentence we manually append the end token
        formatted_sentence = sentence + [END_TOKEN]
        
        # take into account only the END_TOKEN since START token probs are not computed
        word_count += len(formatted_sentence)
        
        # get sentence probability (already measured in log2)
        sum_prob += model.sentence_proba(formatted_sentence) 
    
    return - sum_prob / word_count


def perplexity(cross_entropy: float) -> float:
    """
    Calculate perplexity from cross-entropy.
    
    Perplexity is a measure of how well the language model predicts the given dataset.
    A model with a perplexity of k, has approximately a 1/k chance of correctly predicting the next word in a sentence.
    
    :param cross_entropy: The cross-entropy score calculated for a language model on a dataset.
    :return: The perplexity score as a float.
             Lower values indicate better performance in predicting the dataset.
    """
    return 2**cross_entropy

In [29]:
def ngram_model_alpha_search(fitted_model: BaseNgramModel, 
                             alpha_values: np.ndarray, 
                             validation_dataset: list[list[str]]) -> float:
    """
    Perform alpha hyperparameter search for an n-gram language model.
    
    Given a range of alpha values, evaluate the cross-entropy of the language model
    for each alpha on a validation dataset and return the alpha value that minimizes
    the cross-entropy.
    
    :param fitted_model: The pre-fitted n-gram language model.
    :param alpha_values: Array of alpha values to search through.
    :param validation_dataset: The validation dataset as a list of tokenized sentences,
                               where each sentence is a list of strings.
    :return: The best alpha value that minimizes cross-entropy on the validation dataset.
    """
    entropy_arr = np.full_like(alpha_values, np.inf)
    
    for i in range(len(alpha_values)):
        fitted_model.alpha = alpha_values[i]
        entropy_arr[i] = cross_entropy(fitted_model, validation_dataset)
        
    best_index = np.argmin(entropy_arr)
    return alpha_values[best_index]

(ii). First step: Tune α (alpha)

In [30]:
bi_model = BigramModel(alpha=0.001)
bi_model.fit(train_corpus)

bi_opt_alpha = ngram_model_alpha_search(bi_model, np.linspace(0.0001, 1, 100), development_set)
print("Optimal Bi-gram alpha: ", bi_opt_alpha)

Optimal Bi-gram alpha:  0.010199999999999999


In [31]:
tri_model = TrigramModel(alpha=0.001)
tri_model.fit(train_corpus)

tri_opt_alpha = ngram_model_alpha_search(tri_model, np.linspace(0.0001, 1, 100), development_set)
print("Optimal Tri-gram alpha: ", tri_opt_alpha)

Optimal Tri-gram alpha:  0.010199999999999999


Now, let's test the performance in the test set, after having defined the optimal alpha.

In [32]:
# since la place smoothing is only involved during inference
# we don't need to refit the model
bi_model.alpha = bi_opt_alpha

bi_hc = cross_entropy(bi_model, train_corpus)
print(f"Bi-gram model Cross Entropy: {bi_hc:.2f}", )
print(f"Bi-gram model Perplexity: {perplexity(bi_hc):2f}")

Bi-gram model Cross Entropy: 6.46
Bi-gram model Perplexity: 87.905494


In [33]:
tri_model.alpha = tri_opt_alpha

tri_hc = cross_entropy(tri_model, train_corpus)
print(f"Tri-gram model Cross Entropy: {tri_hc:.2f}")
print(f"Tri-gram model Perplexity: {perplexity(tri_hc):.2f}")

Tri-gram model Cross Entropy: 5.76
Tri-gram model Perplexity: 54.14


In [34]:
# since la place smoothing is only involved during inference
# we don't need to refit the model
bi_model.alpha = bi_opt_alpha

bi_hc = cross_entropy(bi_model, test_set)
print(f"Bi-gram model Cross Entropy: {bi_hc:.2f}", )
print(f"Bi-gram model Perplexity: {perplexity(bi_hc):.2f}")

Bi-gram model Cross Entropy: 7.86
Bi-gram model Perplexity: 232.29


In [35]:
tri_model.alpha = tri_opt_alpha

tri_hc = cross_entropy(tri_model, test_set)
print(f"Tri-gram model Cross Entropy: {tri_hc:.2f}")
print(f"Tri-gram model Perplexity: {perplexity(tri_hc):.2f}")

Tri-gram model Cross Entropy: 8.67
Tri-gram model Perplexity: 407.00


In [36]:
max_depth = 5
beam_width = 3

In [37]:
# You need to give incomplete sentence for autocorrect. Right now it just predicts the end of the sentences after the '.'
# Try a while loop until END_TOKEN is the output maybe

#Check how the auto correction works
predicted = []
for sent in tokenized[:1]:
    print("Original sentence", sent)
    for i, token in enumerate(sent):                     #try all possible combinations within the sentence
        partial_sent = sent[:i+1]
        if END_TOKEN in partial_sent:
            break
        pred = bi_model.predict(tokenized_sentence = partial_sent)
        print("Original sentence: ", partial_sent)
        print("Prediction: ", pred)
        predicted.append(pred)
        print("\n")

Original sentence ['is', 'not', 'my', 'help', 'in', 'me', '?']
Original sentence:  ['is']
Prediction:  written


Original sentence:  ['is', 'not']
Prediction:  hearken


Original sentence:  ['is', 'not', 'my']
Prediction:  dear


Original sentence:  ['is', 'not', 'my', 'help']
Prediction:  believing


Original sentence:  ['is', 'not', 'my', 'help', 'in']
Prediction:  spite


Original sentence:  ['is', 'not', 'my', 'help', 'in', 'me']
Prediction:  saying


Original sentence:  ['is', 'not', 'my', 'help', 'in', 'me', '?']
Prediction:  <end>




## Spell-correcting language models

v. Create a fake dataset

In [38]:
import random


def corrupt_sentence(sentence, probability):
    corrupted_sentence = ""
    for char in sentence:
        if char != ' ' and random.random() < probability:
            
            corrupted_sentence += get_similar_char(char)                 #replace with a similar character
        else:
            corrupted_sentence += char
    return corrupted_sentence


def get_similar_char(char):
    similar_chars = {
        'a': 'e',
        'b': 'p',
        'c': 's',
        'd': 'b',
        'e': 'a',
        'f': 't',
        'g': 'j',
        'h': 'n',
        'i': 'l',
        'j': 'g',
        'k': 'x',
        'l': 'i',
        'm': 'n',
        'n': 'm',
        'o': 'u',
        'p': 'b',
        'q': 'g',
        'r': 't',
        's': 'c',
        't': 'f',
        'u': 'o',
        'v': 'w',
        'w': 'v',
        'x': 'k',
        'y': 'v',
        'z': 's',
    }
    
    return similar_chars.get(char, char)                            #return a randomly chosen character

In [39]:
probability = 0.05                                        #probability of character replacement
corrupted_corpus = [[corrupt_sentence(word, probability) for word in sentence] for sentence in UNK_sentences[:50]]   #generate the corrupted corpus
corrupted_tokenized = [tweet_wt.tokenize(sentence) for sentence_list in corrupted_corpus for sentence in sentence_list] #tokenize the corrupted sentences

for original, corrupted in zip(UNK_sentences[:5], corrupted_corpus[:5]):
    print(f"Original: {original}")
    print(f"Corrupted: {corrupted}")
    print("\n---\n")

Original: ['is not my help in me ?']
Corrupted: ['is not my helb in me ?']

---

Original: ['and they buried him in the border of his inheritance in <UNK> which is in mount ephraim on the north side of the hill of <UNK> .']
Corrupted: ['and they buriab him in tne border of his inheritance in <UNK> whicn is in mount aphrein on the north sibe of the nlll of <UNK> .']

---

Original: ["he 's <UNK> and what 's worse he 's rich ."]
Corrupted: ["he 's <UNK> and what 'c worse he 's rich ."]

---

Original: ['time will generally lessen the interest of every attachment not within the daily circle but that is not the change i had in view for you .']
Corrupted: ['time will generally lescen the interest of every attachment not within the daily circle but that ls not the change i had in view for you .']

---

Original: ['it is all i can do to forgive this .']
Corrupted: ['it is all i can do to forgive this .']

---



In [40]:
from tqdm import tqdm


corrected = []
corrector = BigramSpellCorrector(bi_model, lamda1=0., lamda2=1.)

for sent in tqdm(corrupted_tokenized[:5]):
  output_seq = corrector.spell_correct(original_tokenized_sentence=sent, beam_width=2)  #give the corrupt sentences to spell correct
  corrected.append(output_seq)
    
print('Original sentences:', UNK_sentences[:5])
print('\n')
print('Corrupted(wrong) sentences:', corrupted_corpus[:5])
print('\n')
print('Final result (corrected sentences):', corrected[:5])

100%|█████████████████████████████████████████████| 5/5 [00:09<00:00,  1.89s/it]

Original sentences: [['is not my help in me ?'], ['and they buried him in the border of his inheritance in <UNK> which is in mount ephraim on the north side of the hill of <UNK> .'], ["he 's <UNK> and what 's worse he 's rich ."], ['time will generally lessen the interest of every attachment not within the daily circle but that is not the change i had in view for you .'], ['it is all i can do to forgive this .']]


Corrupted(wrong) sentences: [['is not my helb in me ?'], ['and they buriab him in tne border of his inheritance in <UNK> whicn is in mount aphrein on the north sibe of the nlll of <UNK> .'], ["he 's <UNK> and what 'c worse he 's rich ."], ['time will generally lescen the interest of every attachment not within the daily circle but that ls not the change i had in view for you .'], ['it is all i can do to forgive this .']]


Final result (corrected sentences): [['<start>', 'is', 'not', 'my', 'help', 'in', 'me', '?', '<end>'], ['<start>', 'and', 'they', 'buried', 'him', 'in', '

In [41]:
from jiwer import cer, wer


count = 0
sum_cer = 0
sum_wer = 0

In [43]:
for corrected_sentence in corrected:
    for i in range(len(corrected_sentence)):
        token_k = START_TOKEN if i == 0 else tokenized[i][i - 1]
        token_j = corrected_sentence[i]
        sum_cer += cer(token_k, token_j)
        sum_wer += wer(token_k, token_j)
        count += 1
        
avg_cer = sum_cer/count
avg_wer = sum_wer/count

print(f'Avg cer = {avg_cer}')
print(f'Avg wer = {avg_wer}')

IndexError: list index out of range